In [1]:
import numpy as np
import pandas as pd
import Linear_Regression as LR
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
a_none = np.array(None)
data = pd.read_csv('data.csv')

print(data['price'].describe())
muito_abaixo = data['price'].describe().loc['25%']
abaixo = data['price'].describe().loc['50%']
acima = data['price'].describe().loc['75%']

count       893.000000
mean      79907.409854
std       60880.043823
min        9999.000000
25%       44500.000000
50%       61990.000000
75%       90990.000000
max      450039.000000
Name: price, dtype: float64


In [3]:
# Ajeitando o data set, transformando a variável dependente de continua para discreta
data['preco']=pd.cut(x=data['price'], bins=[0,muito_abaixo,abaixo,acima,np.inf], 
                        labels=["muito_abaixo", "abaixo", "acima","muito_acima"])

l_data = pd.cut(x=data['price'], bins=[0,abaixo,np.inf], 
                        labels=[0, 1])
data = data.drop(['price', 'Unnamed: 0.1',	'Unnamed: 0'], axis=1)
data_c = data[0:500].select_dtypes(include='number')
# Adicionando a primeira coluna com 1 para ser usada como constante
data_c.insert(0, 'constant', [1]*500)
# Ortogonalizando os dados
data_ort = LR.gram_schmidt(data_c)
data_ort['preco'] = data['preco']

In [4]:
# Eliminando as categorias adicionando colunas para representa-las
def one_hot_encode(data_set, column):
    data_set = pd.DataFrame(data_set)
    one_hot = pd.DataFrame(0, index=range(data_set.shape[0]), columns=data_set[column].unique())
    count=0
    for i in data_set[column]:
        one_hot.loc[count, i] = 1
        count+=1
    return pd.concat([data_set.drop(column, axis=1), one_hot], axis=1)

In [5]:
data_class = data_ort.copy()
data_ort = one_hot_encode(data_ort, 'preco')
data_ort

,constant,spec_rating,display_size,resolution_width,resolution_height,warranty,abaixo,muito_abaixo,acima,muito_acima
0,1,4.222706,0.242761,-211.589158,-64.808018,-0.067473,1,0,0,0
1,1,-8.777294,0.773877,98.434168,10.318448,0.004980,0,1,0,0
2,1,0.546235,-1.207036,-149.409749,-114.608444,-0.046807,0,1,0,0
3,1,-2.777294,-1.071253,249.849608,43.403431,-0.018707,1,0,0,0
4,1,0.546235,-1.907036,479.435370,11.919339,-0.033314,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
495,1,0.546235,0.392964,-123.912878,-43.561846,-0.046983,1,0,0,0
496,1,0.546235,-1.207036,-149.409749,-114.608444,-0.046807,1,0,0,0
497,1,0.546235,0.392964,-123.912878,-43.561846,-0.046983,0,1,0,0
498,1,-5.777294,-0.948688,241.393452,146.038974,0.001357,1,0,0,0


Podemos usar a regressão linear já programada para o caso de regressão. Como linear regression prevê o valor y de determinado ponto x tentando maximizar a expectativa de y dado o observado y. Podemos concluir o mesmo para o caso de classificação, tendo uma "probabilidade" de dado a observação x a mesma pertencer a classe i. (probabilidade entre aspas pois os valores podem ser maior que 1 ou menor que 0, apesar de ao serem somados o resultado é 1)

In [6]:
# Função para reunir as estimativas de cada least-square
def LS_categorical(data, colunas):
    # Será salvo cada classe do least-square nessa lista
    lista = []
    # Será salvo a estimativa dos least square para cada observação
    predictions = np.ones([len(data), len(colunas)])
    n_var = len(data.columns) - len(colunas)
    # Será salvo os parâmetros dos least-square
    parametros = np.ones([n_var, len(colunas)])
    j = 0
    for i in colunas:
        y = data[i]
        x = data.drop(colunas, axis=1)
        lr = LR.LR(x,y)
        param = lr.least_square()
        parametros[:,j] = param
        pred = lr.prediction()
        predictions[:,j]=pred
        lista.append(lr)
        j+=1
    # A classe escolhida é aquela em que determinado ls previu o maior valor
    select = np.argmax(predictions, axis=1).reshape(len(data),1)
    return parametros, np.append(predictions, select, axis=1), lista

par, prediction_LS, ls_class = LS_categorical(data_ort, ["muito_abaixo", "abaixo", "acima","muito_acima"])
prediction_LS

array([[ 0.32142942,  0.22468941,  0.21744816,  0.23643301,  0.        ],
       [ 0.39172497,  0.51835731,  0.21641287, -0.12649515,  1.        ],
       [ 0.4065663 ,  0.20588501,  0.24444966,  0.14309903,  0.        ],
       ...,
       [ 0.34130938,  0.30774029,  0.21715537,  0.13379495,  0.        ],
       [ 0.30494999,  0.32648893,  0.23302732,  0.13553375,  1.        ],
       [ 0.34665182,  0.10639524,  0.16915904,  0.3777939 ,  3.        ]])

In [7]:
"Todas as somas de 'probabilidades' resultam em 1"
np.sum(prediction_LS[:, :-1], axis=1)[:5]

array([1., 1., 1., 1., 1.])

In [8]:
'''Acertamos metade das previsões feitas, sendo a média (no chute) 0.25'''
# Passando as categorias para int
cat_int = data_class['preco'].map({"muito_abaixo":0, "abaixo":1, "acima":2,"muito_acima":3})
# Havendo 500 dados previsto
sum(cat_int.to_numpy() - prediction_LS[:,-1] == 0)/500

0.502

Um dos possíveis modelos, criados justamente para o caso de classificação é o linear discriminant analysis, nele é assumido que os dados são:
 - Gaussian distribution
 - A covariação entre as diferentes classes é a mesma(isso permite cancelações tornando o modelo linear em x)

In [9]:
class LDA:
    def __init__(self, data, prediction_col=None):
        self.data = pd.DataFrame(data)
        if prediction_col == None:
            self.x = self.data.iloc[:, :-1]
            self.data.iloc[:,-1] = pd.DataFrame(pd.Categorical(self.data.iloc[:, -1]).codes, columns=['preco'])
            self.y = pd.DataFrame(self.data.iloc[:,-1], columns=['preco'])
        else:
            self.x = self.data.drop(prediction_col, axis=1)
            self.data[prediction_col] = pd.DataFrame(pd.Categorical(self.data[prediction_col]).codes, columns=['preco'])
            self.y = pd.DataFrame(self.data[prediction_col], columns=[prediction_col])
        self.classes = np.array(list(self.y.value_counts(sort=False).values))
        self.n_classes = len(self.classes)
        # Quantia de classes
        self.n_var = len(self.x.columns)

    def parametros(self):
        self.prior = np.ones([self.n_classes, 1])
        for i in range(len(self.prior)):
            self.prior[i] = self.classes[i] / sum(self.classes)
        self.mean = np.ones([self.n_classes, self.n_var])
        for i in range(self.n_classes):
            index = self.y[self.y == i].dropna().index
            self.mean[i] = np.mean(self.x.loc[index], axis=0) 
        self.cov = self.x.cov().to_numpy()
        
    def prediction(self, y=a_none, x=a_none):
        if y.all() == None:
            # Iremos manter a prob de cada classe com a ultima coluna sendo a classe escolhida
            predict = np.ones([len(self.y), self.n_classes+1])
            for i in range(len(self.x)):
                div = 0
                for j in range(self.n_classes):
                    div = div + multivariate_normal.pdf(self.x.loc[i], self.mean[j], self.cov)*self.prior[j]

                # Usado para identificar a classe com maior probabilidade
                prob_aux = 0
                for j in range(self.n_classes):  
                    # Probabilidade 
                    prob = multivariate_normal.pdf(self.x.loc[i], self.mean[j], self.cov)*self.prior[j]/div
                    if prob>prob_aux:
                        prob_aux = prob[0]
                        predict[i, -1] = j
                    predict[i, j] = prob[0]
                    self.predict = predict
            return predict
        else:
            # Iremos manter a prob de cada classe com a ultima coluna sendo a classe escolhida
            predict = np.ones([len(y), self.n_classes+1])
            for i in range(len(x)):
                div = 0
                for j in range(self.n_classes):
                    div = div + multivariate_normal.pdf(x[i], self.mean[j], self.cov)*self.prior[j]
                # Usado para identificar a classe com maior probabilidade
                prob_aux = 0
                for j in range(self.n_classes):  
                    # Probabilidade 
                    prob = multivariate_normal.pdf(x[i], self.mean[j], self.cov)*self.prior[j]/div      
                    if prob>prob_aux:
                        prob_aux = prob[0]
                        predict[i, -1] = j
                    predict[i, j] = prob[0]
            return predict

    def accurency(self, y_hat=a_none, y=a_none):
        if y_hat.all() == None:
            acerto=0
            for i in range(len(self.y)):
                if self.y.loc[i].values[0] - self.predict[i][-1] == 0:
                    acerto+=1      
            return acerto/len(self.y)
        else:
            acerto=0
            y_hat = pd.DataFrame(y_hat)
            y = pd.DataFrame(y)
            for i in range(len(y)):
                if y.loc[i].values[0] - y_hat.loc[i].values[0] == 0:
                    acerto+=1           
            return acerto/len(y)

In [10]:
# Os dados em lda não usam a primeira coluna com 1
lda_data = data_class.iloc[:,1:]
lda = LDA(lda_data,prediction_col='preco')
lda.parametros()
pred = lda.prediction()
print("Porcentagem de acerto LDA: ",lda.accurency())

Porcentagem de acerto LDA:  0.486


Quadratic Discriminant Analysis(QDA) é uma modelo similar a LDA, porém por não conter a suposição que a covariância das diferentes classes sejam iguais não há termos que se cancelem, dessa forma permanece ainda na função um termo quadrático em x, tornando esse modelo não linear

In [11]:
class QDA:
    def __init__(self, data, prediction_col=None):
        self.data = pd.DataFrame(data)
        if prediction_col == None:
            self.x = self.data.iloc[:, :-1]
            self.data.iloc[:,-1] = pd.DataFrame(pd.Categorical(self.data.iloc[:, -1]).codes, columns=['preco'])
            self.y = pd.DataFrame(self.data.iloc[:,-1], columns=['preco'])
        else:
            self.x = self.data.drop(prediction_col, axis=1)
            self.data[prediction_col] = pd.DataFrame(pd.Categorical(self.data[prediction_col]).codes, columns=['preco'])
            self.y = pd.DataFrame(self.data[prediction_col], columns=[prediction_col])
        self.classes = np.array(list(self.y.value_counts(sort=False).values))
        self.n_classes = len(self.classes)
        # Quantia de classes
        self.n_var = len(self.x.columns)

    def parametros(self):
        self.prior = np.ones([self.n_classes, 1])
        for i in range(len(self.prior)):
            self.prior[i] = self.classes[i] / sum(self.classes)
        self.cov = np.ones([self.n_classes, self.n_var, self.n_var])
        self.mean = np.ones([self.n_classes, self.n_var])
        for i in range(self.n_classes):
            index = self.y[self.y == i].dropna().index
            self.mean[i] = np.mean(self.x.loc[index], axis=0)
            self.cov[i] = self.x.loc[index].cov()

    def prediction(self, x=a_none):
        if x.all() == None:
            # Iremos manter a prob de cada classe com a ultima coluna sendo a classe escolhida
            predict = np.ones([len(self.y), self.n_classes+1])
            for i in range(len(self.x)):
                div = 0
                for j in range(self.n_classes):
                    div = div + multivariate_normal.pdf(self.x.loc[i], self.mean[j], self.cov[j])*self.prior[j]
                # Usado para identificar a classe com maior probabilidade
                prob_aux = 0
                for j in range(self.n_classes):  
                    # Probabilidade 
                    prob = multivariate_normal.pdf(self.x.loc[i], self.mean[j], self.cov[j])*self.prior[j]/div
                    if prob>prob_aux:
                        prob_aux = prob[0]
                        predict[i, -1] = j
                    predict[i, j] = prob[0]
                    self.predict = predict
            return predict
        else:
            # Iremos manter a prob de cada classe com a ultima coluna sendo a classe escolhida
            predict = np.ones([len(y), self.n_classes+1])
            for i in range(len(x)):
                div = 0
                for j in range(self.n_classes):
                    div = div + multivariate_normal.pdf(x[i], self.mean[j], self.cov[j])*self.prior[j]
                # Usado para identificar a classe com maior probabilidade
                prob_aux = 0
                for j in range(self.n_classes):  
                    # Probabilidade 
                    prob = multivariate_normal.pdf(x[i], self.mean[j], self.cov[j])*self.prior[j]/div
                    if prob>prob_aux:
                        prob_aux = prob[0]
                        predict[i, -1] = j
                    predict[i, j] = prob[0]
            return predict
        
    def accurency(self, y_hat=a_none, y=a_none):
        if y_hat.all() == None:
            acerto=0
            for i in range(len(self.y)):
                if self.y.loc[i].values[0] - self.predict[i][-1] == 0:
                    acerto+=1
            return acerto/len(self.y)
        else:
            acerto=0
            y_hat = pd.DataFrame(y_hat)
            y = pd.DataFrame(y)
            for i in range(len(y)):
                if y.loc[i].values[0] - y_hat.loc[i].values[0] == 0:
                    acerto+=1           
            return acerto/len(y)

In [12]:
qda = QDA(lda_data,prediction_col='preco')
qda.parametros()
pred_qda = qda.prediction()
print("Porcentagem de acerto QDA: ",qda.accurency())

Porcentagem de acerto QDA:  0.45


In [13]:
class Logistic_Regression:
    def __init__(self, data, prediction_col=None):
        self.data = pd.DataFrame(data)
        if prediction_col == None:
            self.x = self.data.iloc[:, :-1].to_numpy()
            self.data.iloc[:,-1] = pd.DataFrame(pd.Categorical(self.data.iloc[:, -1]).codes, columns=['preco'])
            self.y = pd.DataFrame(self.data.iloc[:,-1], columns=['preco']).to_numpy()            
        else:
            self.x = self.data.drop(prediction_col, axis=1).to_numpy()
            self.data[prediction_col] = pd.DataFrame(pd.Categorical(self.data[prediction_col]).codes, columns=['preco'])
            self.y = pd.DataFrame(self.data[prediction_col], columns=[prediction_col]).to_numpy()
        self.classes = np.array(list(np.unique(self.y)))
        self.n_classes = len(self.classes)

    def treino(self):
        self.parametros = np.array([0]*(self.x.shape[1])).reshape(self.x.shape[1])
        p1_passado = 0
        p1=np.inf
        while abs(np.sum(p1_passado - p1)) > 0.0001:
            # Manter as mudanças nas estimativas
            p1_passado = p1
            self.parametros = self.parametros.reshape(self.x.shape[1], 1) 
            # Calculando a probabilidade de cada observação
            numerador = np.exp(self.x.dot(self.parametros))
            denominador = (1 + np.exp(self.x.dot(self.parametros)))
            p1 = (numerador/denominador).reshape(self.x.shape[0], 1)
            p2 = 1-p1
            # Iniciando as matrizes que irão manter as derivadas 1 e 2
            der_b = np.zeros([1, self.x.shape[1]])
            der_b2 = np.zeros([self.x.shape[1], self.x.shape[1]])           
            for i in range(self.x.shape[0]):
                der_b2 = der_b2 + (self.x[i].reshape(self.x.shape[1],1).dot(self.x[i].reshape(1,self.x.shape[1])))*p1[i]*p2[i]
                der_b = der_b + self.x[i]*((self.y[i] - p1[i]))  
            der_b2 *=-1
            b_new = self.parametros - np.linalg.inv(der_b2).dot(der_b.T)
            self.parametros = b_new
        return self.parametros 

    def prediction(self, x=a_none):
        if x.all() == None:
            numerador = np.exp(self.x.dot(self.parametros))
            denominador = (1 + np.exp(self.x.dot(self.parametros)))
            p1 = (numerador/denominador).reshape(self.x.shape[0], 1)
            pred = np.append(p1, np.where(p1 > 0.5,1,0), axis=1)
            self.predict = pred
            return pred
        else:
            numerador = np.exp(x.dot(self.parametros))
            denominador = (1 + np.exp(x.dot(self.parametros)))
            p1 = (numerador/denominador).reshape(x.shape[0], 1)
            pred = np.append(p1, np.where(p1 > 0.5,1,0), axis=1)
            return pred
        
    def accurency(self, y_hat=a_none, y=a_none):
        if y_hat.all() == None:
            acerto=0
            predict = self.predict[:,-1]
            for i in range(len(self.y)):
                if (self.y[i] - predict[i])[0] == 0:
                    acerto+=1
            return acerto/len(self.y)
        else:
            acerto=0
            for i in range(len(y)):
                if (y[i]- y_hat[i])[0] == 0:
                    acerto+=1           
            return acerto/len(y)   

In [14]:
lda_data['pr'] = l_data
logistic_data = pd.concat([pd.DataFrame(np.ones([500,1]), columns=['constant']), lda_data],  axis=1)
logistic_data.drop('preco', axis=1, inplace=True)
lr = Logistic_Regression(logistic_data,prediction_col='pr')
lr.treino()
lr.prediction()
lr.accurency()

0.782

In [3]:
class Perceptron:
    def __init__(self, data):
        self.data = data
        self.x = data[:,:-1]
        self.y = data[:,-1]
        self.parametros = np.ones([self.x.shape[1], 1])
        
    def predict(self, x=a_none):
        if x.all() == None:
            predict = self.x.dot(self.parametros)
            self.pred_binary = np.append(predict, np.where(predict > 0, 1, -1), axis=1)
            return self.pred_binary
        else:
            x = np.array(x)
            predict = x.dot(self.parametros)
            pred_binary = np.append(predict, np.where(predict > 0, 1, -1), axis=1)
            return pred_binary

    def treino(self, p=1):
        erro = np.inf
        self.x_erro = []
        i = 0
        best_param = self.parametros.copy() 
        for i in range(1000):
            pred_binary = self.predict()
            lugar_erro = (self.y - pred_binary[:,1]) != 0
            pred_erro = pred_binary[lugar_erro]
            x_erro = self.x[lugar_erro]
            
            if len(x_erro) == 0:
                break
            elif len(x_erro) < erro:
                # Mantendo os melhores parâmetros encontrados
                erro = len(x_erro)
                best_param = self.parametros.copy() 
            elif abs(self.parametros[0]) > 1e+50 or abs(self.parametros[0]) < 1e-50:
                self.parametros = best_param.copy() 
                return best_param
            
            # Escolher uma observação errada aleatóriamente
            obs = np.random.randint(0, len(pred_erro))
            # Calculando a derivada dos parâmetros
            der_b = x_erro[obs,1:]*pred_erro[obs, 0]
            der_b0 = pred_erro[obs,0]
            der_b = -der_b
            der_b0 = -der_b0
            # Fazer o update dos parâmetros
            self.parametros[0] = self.parametros[0] + p*der_b0
            self.parametros[1:] = self.parametros[1:] + p*der_b.reshape([len(der_b), 1])

        self.parametros = best_param
        return self.parametros
    # Encontrar os pontos em 2d para trassar uma linha
    def line(self):
        coor = np.ones([2,2])
        coor[0,0] = np.min(self.data[:,1])
        coor[0,1] = (-self.parametros[0] - self.parametros[1]*coor[0,0])/self.parametros[2]

        coor[1,0] = np.max(self.data[:,1])
        coor[1,1] = (-self.parametros[0] -self.parametros[1]*coor[1,0])/self.parametros[2]
        self.coor = coor

    def acurancy(self, y=a_none, x=a_none):
        if x.all()==None:
            return sum((self.predict()[:,1] - self.y == 0))/len(self.x)
        else:
            return sum((self.predict(x)[:,1] - y == 0))/len(x)